In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, top_k_accuracy_score
from joblib import dump
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import numpy as np
import xgboost as xgb

In [92]:
# Read files
train_df = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
original_df = pd.read_csv('/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv')

In [93]:
# Mark categorical columns
for col in test_df.select_dtypes(include=['object']).columns:
    train_df[col] = train_df[col].astype('category')
    original_df[col] = original_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

In [94]:
train_df.pop('id')
train_target = train_df.pop('Fertilizer Name')
train_df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,37,70,36,Clayey,Sugarcane,36,4,5
1,27,69,65,Sandy,Millets,30,6,18
2,29,63,32,Sandy,Millets,24,12,16
3,35,62,54,Sandy,Barley,39,12,4
4,35,58,43,Red,Paddy,37,2,16


In [95]:
original_target = original_df.pop('Fertilizer Name')
original_df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,32,51,41,Red,Ground Nuts,7,3,19
1,35,58,35,Black,Cotton,4,14,16
2,27,55,43,Sandy,Sugarcane,28,0,17
3,33,56,56,Loamy,Ground Nuts,37,5,24
4,32,70,60,Red,Ground Nuts,4,6,9


In [96]:
test_df
test_df.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


## Preprocessing

In [97]:
X = pd.concat([train_df, original_df])
X

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,37,70,36,Clayey,Sugarcane,36,4,5
1,27,69,65,Sandy,Millets,30,6,18
2,29,63,32,Sandy,Millets,24,12,16
3,35,62,54,Sandy,Barley,39,12,4
4,35,58,43,Red,Paddy,37,2,16
...,...,...,...,...,...,...,...,...
99995,32,71,61,Black,Tobacco,23,1,25
99996,35,72,47,Loamy,Millets,38,1,32
99997,28,50,61,Sandy,Maize,10,11,14
99998,29,57,63,Loamy,Ground Nuts,7,10,4


In [98]:
y = pd.concat([train_target, original_target])
y

0           28-28
1           28-28
2        17-17-17
3        10-26-26
4             DAP
           ...   
99995       20-20
99996    17-17-17
99997    14-35-14
99998         DAP
99999    17-17-17
Name: Fertilizer Name, Length: 850000, dtype: object

In [99]:
X_test = test_df.copy()
X_test.pop('id')

0         750000
1         750001
2         750002
3         750003
4         750004
           ...  
249995    999995
249996    999996
249997    999997
249998    999998
249999    999999
Name: id, Length: 250000, dtype: int64

In [100]:
# Encode target labels
le = LabelEncoder()
y = le.fit_transform(y)

## Finetune

In [104]:
params = {
        'objective': 'multi:softprob',
        'num_class': 7,
        'max_depth': 16,
        'learning_rate': 0.01,
        'reg_alpha': 3,
        'reg_lambda': 1.4,
        'gamma': 0.26,
        'max_delta_step': 5,
        'subsample': 0.86,
        'colsample_bytree': 0.4,
        'min_child_weight': 5,
        'random_state': 42,
        'n_jobs': -1,
        'eval_metric': 'mlogloss',
        'device': "cuda"   
}

In [105]:
# Create ensemble
folds = 5
skf = StratifiedKFold(n_splits= folds, shuffle=True, random_state=42)

test_probs = np.zeros((len(X_test), len(le.classes_)))

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    
    print(f"Fold {fold+1}:")
    
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train_fold, label=y_train_fold, enable_categorical=True)
    dval = xgb.DMatrix(X_val_fold, label=y_val_fold, enable_categorical=True)
    dtest = xgb.DMatrix(X_test, enable_categorical=True)

    xgb_model_fold = xgb.train(
        params, 
        dtrain, 
        num_boost_round=10_000, 
        evals=[(dtrain, 'train'), (dval, 'validation')], 
        early_stopping_rounds=50, 
        verbose_eval=100
    )

    # Predict test set
    test_probs_fold = xgb_model_fold.predict(dtest)
    test_probs += test_probs_fold / skf.n_splits

Fold 1:
[0]	train-mlogloss:1.94569	validation-mlogloss:1.94579
[100]	train-mlogloss:1.92334	validation-mlogloss:1.93537
[200]	train-mlogloss:1.90457	validation-mlogloss:1.92771
[300]	train-mlogloss:1.88767	validation-mlogloss:1.92164
[400]	train-mlogloss:1.87173	validation-mlogloss:1.91659
[500]	train-mlogloss:1.85704	validation-mlogloss:1.91236
[600]	train-mlogloss:1.84373	validation-mlogloss:1.90880
[700]	train-mlogloss:1.83091	validation-mlogloss:1.90573
[800]	train-mlogloss:1.81897	validation-mlogloss:1.90302
[900]	train-mlogloss:1.80758	validation-mlogloss:1.90068
[1000]	train-mlogloss:1.79685	validation-mlogloss:1.89863
[1100]	train-mlogloss:1.78667	validation-mlogloss:1.89685
[1200]	train-mlogloss:1.77693	validation-mlogloss:1.89522
[1300]	train-mlogloss:1.76798	validation-mlogloss:1.89384
[1400]	train-mlogloss:1.75917	validation-mlogloss:1.89257
[1500]	train-mlogloss:1.75072	validation-mlogloss:1.89150
[1600]	train-mlogloss:1.74345	validation-mlogloss:1.89062
[1700]	train-mlogl

## Create Submission

In [68]:
# Get top 3 predicted labels from averaged probabilities
test_top3_idx = test_probs.argsort(axis=1)[:, -3:][:, ::-1]
test_top3_labels = le.inverse_transform(test_top3_idx.ravel()).reshape(test_top3_idx.shape)

In [90]:
# Save test predictions from ensemble
top3_str = [' '.join(row) for row in test_top3_labels]
submission = pd.DataFrame({
    'id': test_df['id'],
    'Fertilizer Name': top3_str
})
submission.to_csv("submission.csv", index=False)
display(submission.head())

,id,Fertilizer Name
0,750000,10-26-26 28-28 14-35-14
1,750001,17-17-17 20-20 10-26-26
2,750002,20-20 10-26-26 14-35-14
3,750003,14-35-14 10-26-26 17-17-17
4,750004,20-20 10-26-26 28-28
